In [ ]:
!pip install -U -q pydrive google-colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

# Authenticate and mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results2"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate=0.2, dense_units=128):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate=0.2, lstm_units=64):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate=0.2, filters=64):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")


Size of DataFrame after dropping null values: (4958, 8)
Training MLP...
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7070 - loss: 0.6495 - val_accuracy: 0.7994 - val_loss: 0.4507
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8067 - loss: 0.5036 - val_accuracy: 0.8075 - val_loss: 0.4578
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8089 - loss: 0.4938 - val_accuracy: 0.8135 - val_loss: 0.4263
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8163 - loss: 0.4820 - val_accuracy: 0.8075 - val_loss: 0.4538
Epoch 5/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8235 - loss: 0.4737 - val_accuracy: 0.8105 - val_loss: 0.4440
Epoch 6/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8373 - loss: 0.4499 - val_accuracy: 0.8004 - val_loss: 0.4636
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7215 - loss: 0.6177 - val_accuracy: 0.7087 - val_loss: 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results5"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate, dense_units):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate, lstm_units):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate, filters):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")

Size of DataFrame after dropping null values: (4958, 8)
Training MLP...
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7140 - loss: 0.6325 - val_accuracy: 0.8095 - val_loss: 0.4467
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8067 - loss: 0.5083 - val_accuracy: 0.7873 - val_loss: 0.4730
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8233 - loss: 0.4815 - val_accuracy: 0.7913 - val_loss: 0.4774
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8133 - loss: 0.4765 - val_accuracy: 0.7792 - val_loss: 0.4890
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.7075 - loss: 0.6428 - val_accuracy: 0.7903 - val_loss: 0.4430
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8010 - loss: 0.5273 - val_accuracy: 0.8034 - val_loss: 0.4346
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8219 - loss: 0.4804 - val_accuracy: 0.7994 - val_loss

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results6"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate, dense_units):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate, lstm_units):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate, filters):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")

Size of DataFrame after dropping null values: (4958, 8)
Training MLP...
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7410 - loss: 0.6352 - val_accuracy: 0.8034 - val_loss: 0.4331
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7975 - loss: 0.5205 - val_accuracy: 0.8054 - val_loss: 0.4397
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8096 - loss: 0.4943 - val_accuracy: 0.8044 - val_loss: 0.4558
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8306 - loss: 0.4660 - val_accuracy: 0.8054 - val_loss: 0.4368
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7369 - loss: 0.6140 - val_accuracy: 0.7712 - val_loss: 0.4973
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7986 - loss: 0.5121 - val_accuracy: 0.7712 - val_loss: 0.4900
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8150 - loss: 0.4867 - val_accuracy: 0.8054 - val_los

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results7"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate, dense_units):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate, lstm_units):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate, filters):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20, 30],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128, 256]
    },
    'LSTM': {
        'epochs': [10, 20, 30],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128, 256],
    },
    'CNN': {
        'epochs': [10, 20, 30],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128, 256],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx'

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results8"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate, dense_units):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate, lstm_units):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate, filters):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")

Size of DataFrame after dropping null values: (4958, 8)
Training MLP...
Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.7356 - loss: 0.6292 - val_accuracy: 0.8095 - val_loss: 0.4460
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7926 - loss: 0.5303 - val_accuracy: 0.8125 - val_loss: 0.4331
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8231 - loss: 0.4782 - val_accuracy: 0.7954 - val_loss: 0.4599
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8159 - loss: 0.4685 - val_accuracy: 0.8034 - val_loss: 0.4321
Epoch 5/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8114 - loss: 0.4688 - val_accuracy: 0.8044 - val_loss: 0.4314
Epoch 6/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8309 - loss: 0.4485 - val_accuracy: 0.7067 - val_loss: 0.5760
Epoch 7/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8269 - loss: 0.4498 - val_accuracy: 0.7913 - val_loss: 0.4936
Epoch 8/10
248/248 ━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized1"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
    except:
        pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
embedding_dim = 768

# Apply parsing to embeddings
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, embedding_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, embedding_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        Dropout(0.4),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        Dropout(0.4),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}
for name, model in models.items():
    print(f"Training {name}...")
    if name == 'MLP':
        model.fit(X_train_flattened, y_train, epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test), callbacks=[early_stopping], verbose=1)
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model
    model.save(os.path.join(save_path, f"{name}_model.h5"))

    # Evaluate model
    results[name] = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
    }

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

print("Training complete. Models and results saved.")


Training MLP...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.7113 - loss: 0.6780 - val_accuracy: 0.7903 - val_loss: 0.5133
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8074 - loss: 0.4249 - val_accuracy: 0.7893 - val_loss: 0.4742
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8084 - loss: 0.4094 - val_accuracy: 0.8004 - val_loss: 0.4402
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8264 - loss: 0.3906 - val_accuracy: 0.8034 - val_loss: 0.4523
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8290 - loss: 0.3728 - val_accuracy: 0.8014 - val_loss: 0.4337
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8374 - loss: 0.3610 - val_accuracy: 0.8115 - val_loss: 0.4437
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8287 - loss: 0.3657 - val_accuracy: 0.8145 - val_loss: 0.4269
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8371 - loss: 0.3494 -

Training LSTM...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 9s 53ms/step - accuracy: 0.5600 - loss: 0.6850 - val_accuracy: 0.6048 - val_loss: 0.6646
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.6160 - loss: 0.6519 - val_accuracy: 0.6079 - val_loss: 0.6582
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.6039 - loss: 0.6560 - val_accuracy: 0.6331 - val_loss: 0.6521
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - accuracy: 0.6288 - loss: 0.6404 - val_accuracy: 0.6321 - val_loss: 0.6416
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - accuracy: 0.6442 - loss: 0.6315 - val_accuracy: 0.6290 - val_loss: 0.6400
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.6616 - loss: 0.6243 - val_accuracy: 0.6532 - val_loss: 0.6310
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - accuracy: 0.6331 - loss: 0.6359 - val_accuracy: 0.6200 - val_loss: 0.6575
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.6174 - los

Training CNN...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.4974 - loss: 0.6925 - val_accuracy: 0.6139 - val_loss: 0.6785
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6233 - loss: 0.6758 - val_accuracy: 0.6704 - val_loss: 0.6469
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6867 - loss: 0.6424 - val_accuracy: 0.6784 - val_loss: 0.6100
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6887 - loss: 0.6080 - val_accuracy: 0.6865 - val_loss: 0.5916
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7209 - loss: 0.5762 - val_accuracy: 0.6946 - val_loss: 0.5799
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7234 - loss: 0.5636 - val_accuracy: 0.7036 - val_loss: 0.5803
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7041 - loss: 0.5731 - val_accuracy: 0.7036 - val_loss: 0.5626
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7340 - loss: 0.5650 -

Training complete. Models and results saved.


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized2"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
    except:
        pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
embedding_dim = 768

# Apply parsing to embeddings
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, embedding_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, embedding_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        Dropout(0.4),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        Dropout(0.4),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}
for name, model in models.items():
    print(f"Training {name}...")
    if name == 'MLP':
        model.fit(X_train_flattened, y_train, epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test), callbacks=[early_stopping], verbose=1)
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model
    model.save(os.path.join(save_path, f"{name}_model.h5"))

    # Evaluate model
    results[name] = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
    }

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

print("Training complete. Models and results saved.")

Training MLP...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.7157 - loss: 0.5598 - val_accuracy: 0.7500 - val_loss: 0.5314
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8133 - loss: 0.4172 - val_accuracy: 0.8105 - val_loss: 0.4517
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8017 - loss: 0.4259 - val_accuracy: 0.8115 - val_loss: 0.4395
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8226 - loss: 0.3900 - val_accuracy: 0.7883 - val_loss: 0.4616
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8364 - loss: 0.3683 - val_accuracy: 0.8095 - val_loss: 0.4340
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8381 - loss: 0.3618 - val_accuracy: 0.8185 - val_loss: 0.4256
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8420 - loss: 0.3592 - val_accuracy: 0.7984 - val_loss: 0.4481
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8425 - loss: 0.3411 -

Training LSTM...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.5458 - loss: 0.6849 - val_accuracy: 0.5978 - val_loss: 0.6648
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.6267 - loss: 0.6518 - val_accuracy: 0.6532 - val_loss: 0.6368
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - accuracy: 0.6356 - loss: 0.6482 - val_accuracy: 0.6411 - val_loss: 0.6365
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.6567 - loss: 0.6205 - val_accuracy: 0.6058 - val_loss: 0.6893
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.5778 - loss: 0.6855 - val_accuracy: 0.5978 - val_loss: 0.6598
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.6345 - loss: 0.6437 - val_accuracy: 0.6562 - val_loss: 0.6223
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.6731 - loss: 0.6097 - val_accuracy: 0.6401 - val_loss: 0.6425
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.6441 - lo

Training CNN...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.5131 - loss: 0.6923 - val_accuracy: 0.6462 - val_loss: 0.6791
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6332 - loss: 0.6765 - val_accuracy: 0.6532 - val_loss: 0.6553
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6730 - loss: 0.6456 - val_accuracy: 0.6754 - val_loss: 0.6152
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7051 - loss: 0.6067 - val_accuracy: 0.6774 - val_loss: 0.5900
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7057 - loss: 0.5862 - val_accuracy: 0.6976 - val_loss: 0.5750
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7154 - loss: 0.5735 - val_accuracy: 0.7117 - val_loss: 0.5631
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7272 - loss: 0.5503 - val_accuracy: 0.7067 - val_loss: 0.5628
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7222 - loss: 0.5539 -

Training complete. Models and results saved.


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized3"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0007), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=30, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")



Training MLP Model...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.7338 - loss: 0.5284 - val_accuracy: 0.8024 - val_loss: 0.4517 - learning_rate: 7.0000e-04
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.7940 - loss: 0.4492 - val_accuracy: 0.7782 - val_loss: 0.4937 - learning_rate: 7.0000e-04
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8088 - loss: 0.4289 - val_accuracy: 0.8004 - val_loss: 0.4371 - learning_rate: 7.0000e-04
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8113 - loss: 0.4144 - val_accuracy: 0.8135 - val_loss: 0.4236 - learning_rate: 7.0000e-04
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8278 - loss: 0.3868 - val_accuracy: 0.8155 - val_loss: 0.4277 - learning_rate: 7.0000e-04
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8158 - loss: 0.4040 - val_accuracy: 0.8065 - val_loss: 0.4216 - learning_rate: 7.0000e-04
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized4"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0007), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=30, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")



Training MLP Model...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.7186 - loss: 0.5595 - val_accuracy: 0.7823 - val_loss: 0.4663 - learning_rate: 7.0000e-04
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7904 - loss: 0.4532 - val_accuracy: 0.8044 - val_loss: 0.4461 - learning_rate: 7.0000e-04
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7881 - loss: 0.4422 - val_accuracy: 0.8054 - val_loss: 0.4405 - learning_rate: 7.0000e-04
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.8049 - loss: 0.4197 - val_accuracy: 0.7994 - val_loss: 0.4823 - learning_rate: 7.0000e-04
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.8167 - loss: 0.4096 - val_accuracy: 0.8085 - val_loss: 0.4444 - learning_rate: 7.0000e-04
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8255 - loss: 0.3893 - val_accuracy: 0.8135 - val_loss: 0.4271 - learning_rate: 7.0000e-04
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized5"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0007), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=30, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")


Training MLP Model...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.7322 - loss: 0.5567 - val_accuracy: 0.7661 - val_loss: 0.4933 - learning_rate: 7.0000e-04
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7872 - loss: 0.4532 - val_accuracy: 0.8125 - val_loss: 0.4495 - learning_rate: 7.0000e-04
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7983 - loss: 0.4433 - val_accuracy: 0.8034 - val_loss: 0.4318 - learning_rate: 7.0000e-04
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8041 - loss: 0.4295 - val_accuracy: 0.8095 - val_loss: 0.4306 - learning_rate: 7.0000e-04
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8190 - loss: 0.4127 - val_accuracy: 0.8125 - val_loss: 0.4364 - learning_rate: 7.0000e-04
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8234 - loss: 0.4100 - val_accuracy: 0.7984 - val_loss: 0.4403 - learning_rate: 7.0000e-04
Epoch 7/30
118/124 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized6"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0007), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=30, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")


Training MLP Model...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.7012 - loss: 0.6013 - val_accuracy: 0.6956 - val_loss: 0.5470 - learning_rate: 7.0000e-04
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7943 - loss: 0.4382 - val_accuracy: 0.7692 - val_loss: 0.4774 - learning_rate: 7.0000e-04
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8084 - loss: 0.4337 - val_accuracy: 0.7923 - val_loss: 0.4729 - learning_rate: 7.0000e-04
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8121 - loss: 0.4016 - val_accuracy: 0.8034 - val_loss: 0.4520 - learning_rate: 7.0000e-04
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8256 - loss: 0.3939 - val_accuracy: 0.8165 - val_loss: 0.4385 - learning_rate: 7.0000e-04
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8171 - loss: 0.3980 - val_accuracy: 0.8054 - val_loss: 0.4375 - learning_rate: 7.0000e-04
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized7"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0007), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=30, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")


Training MLP Model...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.7046 - loss: 0.5765 - val_accuracy: 0.7651 - val_loss: 0.4845 - learning_rate: 7.0000e-04
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7895 - loss: 0.4631 - val_accuracy: 0.7812 - val_loss: 0.4712 - learning_rate: 7.0000e-04
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8113 - loss: 0.4227 - val_accuracy: 0.7782 - val_loss: 0.5069 - learning_rate: 7.0000e-04
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8086 - loss: 0.4143 - val_accuracy: 0.7954 - val_loss: 0.4489 - learning_rate: 7.0000e-04
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8023 - loss: 0.4238 - val_accuracy: 0.8145 - val_loss: 0.4424 - learning_rate: 7.0000e-04
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8140 - loss: 0.4102 - val_accuracy: 0.8085 - val_loss: 0.4287 - learning_rate: 7.0000e-04
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized8"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(256, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0007), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=30, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=30, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")


Training MLP Model...
Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.7106 - loss: 0.5703 - val_accuracy: 0.7944 - val_loss: 0.4403 - learning_rate: 7.0000e-04
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7766 - loss: 0.4765 - val_accuracy: 0.8044 - val_loss: 0.4356 - learning_rate: 7.0000e-04
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8086 - loss: 0.4265 - val_accuracy: 0.7994 - val_loss: 0.4359 - learning_rate: 7.0000e-04
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8142 - loss: 0.4120 - val_accuracy: 0.8065 - val_loss: 0.4413 - learning_rate: 7.0000e-04
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8156 - loss: 0.3978 - val_accuracy: 0.8115 - val_loss: 0.4344 - learning_rate: 7.0000e-04
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8232 - loss: 0.3964 - val_accuracy: 0.8014 - val_loss: 0.4455 - learning_rate: 7.0000e-04
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized9"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(512, activation='relu'),
        LayerNormalization(),
        Dropout(0.4),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(256, return_sequences=True),
        LayerNormalization(),
        Dropout(0.4),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(256, kernel_size=5, activation='relu'),
        LayerNormalization(),
        Dropout(0.4),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=50, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=50, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")



Training MLP Model...
Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.6428 - loss: 0.6485 - val_accuracy: 0.7792 - val_loss: 0.4660 - learning_rate: 5.0000e-04
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7657 - loss: 0.4895 - val_accuracy: 0.8105 - val_loss: 0.4495 - learning_rate: 5.0000e-04
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7908 - loss: 0.4554 - val_accuracy: 0.8044 - val_loss: 0.4394 - learning_rate: 5.0000e-04
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7827 - loss: 0.4656 - val_accuracy: 0.8044 - val_loss: 0.4382 - learning_rate: 5.0000e-04
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8013 - loss: 0.4206 - val_accuracy: 0.8075 - val_loss: 0.4447 - learning_rate: 5.0000e-04
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8056 - loss: 0.4319 - val_accuracy: 0.8065 - val_loss: 0.4333 - learning_rate: 5.0000e-04
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized10"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(512, activation='relu'),
        LayerNormalization(),
        Dropout(0.4),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(256, return_sequences=True),
        LayerNormalization(),
        Dropout(0.4),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(256, kernel_size=5, activation='relu'),
        LayerNormalization(),
        Dropout(0.4),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=50, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=50, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")


Training MLP Model...
Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.6797 - loss: 0.6035 - val_accuracy: 0.7923 - val_loss: 0.4534 - learning_rate: 5.0000e-04
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7893 - loss: 0.4600 - val_accuracy: 0.8085 - val_loss: 0.4476 - learning_rate: 5.0000e-04
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7785 - loss: 0.4547 - val_accuracy: 0.8004 - val_loss: 0.4367 - learning_rate: 5.0000e-04
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7904 - loss: 0.4384 - val_accuracy: 0.7913 - val_loss: 0.4401 - learning_rate: 5.0000e-04
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7905 - loss: 0.4469 - val_accuracy: 0.8075 - val_loss: 0.4368 - learning_rate: 5.0000e-04
Epoch 6/50
119/124 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7944 - loss: 0.4440
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, BatchNormalization, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib
import json

# Enable mixed precision for speed improvement
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results_optimized11"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path).dropna()

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# Define embedding dimension
embedding_dim = 768

# Convert embeddings into arrays
embedding_features = ['word2vec_vector']
for feature in embedding_features:
    df[feature] = df[feature].map(lambda x: parse_vector(x, embedding_dim))

# Stack feature vectors efficiently
X = np.stack(df[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Optimized Model Architectures
def build_mlp(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(512, activation='relu'),
        LayerNormalization(),
        Dropout(0.4),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(256, return_sequences=True),
        LayerNormalization(),
        Dropout(0.4),
        LSTM(128, return_sequences=True),
        LayerNormalization(),
        Dropout(0.3),
        LSTM(64),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(256, kernel_size=5, activation='relu'),
        LayerNormalization(),
        Dropout(0.4),
        Conv1D(128, kernel_size=3, activation='relu'),
        LayerNormalization(),
        Dropout(0.3),
        Conv1D(64, kernel_size=3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for training efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train and save models
models = {
    'MLP': build_mlp((X_train_flattened.shape[1],)),
    'LSTM': build_lstm((X_train.shape[1], X_train.shape[2])),
    'CNN': build_cnn((X_train.shape[1], X_train.shape[2]))
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name} Model...")

    if name == 'MLP':
        history = model.fit(
            X_train_flattened, y_train,
            epochs=50, batch_size=32, validation_data=(X_test_flattened, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test_flattened) > 0.5).astype(int)
    else:
        history = model.fit(
            X_train, y_train,
            epochs=50, batch_size=32, validation_data=(X_test, y_test),
            callbacks=[early_stopping, reduce_lr], verbose=1
        )
        y_pred = (model.predict(X_test) > 0.5).astype(int)

    # Save the trained model using TensorFlow's SavedModel format
    model.save(os.path.join(save_path, f"{name}_model.keras"))

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).tolist()

    results[name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

    print(f"\n{name} Model Evaluation:\nAccuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save evaluation results
with open(os.path.join(save_path, "model_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# Save results in CSV format for easy analysis
pd.DataFrame(results).T.to_csv(os.path.join(save_path, "model_results.csv"), index=True)

print("\nTraining complete. Models and results saved.")


Training MLP Model...
Epoch 1/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.5425 - loss: 0.7640 - val_accuracy: 0.6522 - val_loss: 0.6212 - learning_rate: 5.0000e-04
Epoch 2/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.6003 - loss: 0.6709 - val_accuracy: 0.6643 - val_loss: 0.6205 - learning_rate: 5.0000e-04
Epoch 3/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6207 - loss: 0.6490 - val_accuracy: 0.6774 - val_loss: 0.6117 - learning_rate: 5.0000e-04
Epoch 4/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.6176 - loss: 0.6442 - val_accuracy: 0.6905 - val_loss: 0.6067 - learning_rate: 5.0000e-04
Epoch 5/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6379 - loss: 0.6172 - val_accuracy: 0.6825 - val_loss: 0.5992 - learning_rate: 5.0000e-04
Epoch 6/50
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6285 - loss: 0.6197 - val_accuracy: 0.6724 - val_loss: 0.6031 - learning_rate: 5.0000e-04
Epoch 7/50
124/124 ━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


124/124 ━━━━━━━━━━━━━━━━━━━━ 4218s 34s/step - accuracy: 0.5133 - loss: 0.9741 - val_accuracy: 0.4950 - val_loss: 0.6928 - learning_rate: 3.0000e-04
Epoch 2/50
108/124 ━━━━━━━━━━━━━━━━━━━━ 9:07 34s/step - accuracy: 0.4948 - loss: 0.7148

In [ ]:
refer above

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results4"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Normalize to 768
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Reshape X for BiLSTM (samples, time_steps, features)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define BiLSTM Model
def build_bilstm(input_shape, lstm_units=128, dropout_rate=0.3):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(lstm_units, return_sequences=True)),
        Dropout(dropout_rate),
        Bidirectional(LSTM(lstm_units // 2)),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train BiLSTM model
bilstm_model = build_bilstm((X_train.shape[1], X_train.shape[2]))
bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test),
                 callbacks=[early_stopping], verbose=1)

# Predict and evaluate
y_pred = (bilstm_model.predict(X_test) > 0.5).astype(int)

# Define evaluation function
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Save results
results = evaluate_model(y_test, y_pred)
results_path = os.path.join(save_path, "bilstm_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

# Save model
bilstm_model.save(os.path.join(save_path, "bilstm_model.h5"))

print("Experiment completed. Results saved.")


Size of DataFrame after dropping null values: (4958, 8)
Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 470s 4s/step - accuracy: 0.5513 - loss: 0.6755 - val_accuracy: 0.6169 - val_loss: 0.6525
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 484s 4s/step - accuracy: 0.6078 - loss: 0.6474 - val_accuracy: 0.6485 - val_loss: 0.6340
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 532s 4s/step - accuracy: 0.6272 - loss: 0.6312 - val_accuracy: 0.6855 - val_loss: 0.5911
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 502s 4s/step - accuracy: 0.6927 - loss: 0.5822 - val_accuracy: 0.6902 - val_loss: 0.5934
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 499s 4s/step - accuracy: 0.6813 - loss: 0.5828 - val_accuracy: 0.6250 - val_loss: 0.6502
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 504s 4s/step - accuracy: 0.6886 - loss: 0.5821 - val_accuracy: 0.7527 - val_loss: 0.5187
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 468s 4s/step - accuracy: 0.6929 - loss: 0.5689 - val_accuracy: 0.7487 - val_loss: 0.5280
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 4

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Input, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
from tensorflow.keras.callbacks import EarlyStopping

# Set paths
data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/data.xlsx"
save_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/tested_models/deep_learning_results2"
os.makedirs(save_path, exist_ok=True)

# Load dataset
df = pd.read_excel(data_path)
df.dropna(inplace=True)
print("Size of DataFrame after dropping null values:", df.shape)

# Function to parse vector strings into numpy arrays
def parse_vector(vector_str, expected_dim):
    try:
        if isinstance(vector_str, str):
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            if vec.shape[0] == expected_dim:
                return vec
            elif vec.shape[0] > expected_dim:
                return vec[:expected_dim]  # Truncate to expected size
            else:
                return np.pad(vec, (0, expected_dim - vec.shape[0]))  # Pad with zeros
    except:
        return np.zeros(expected_dim, dtype=np.float32)  # Default to zero vector
    return np.zeros(expected_dim, dtype=np.float32)

# Define expected dimensions
word2vec_dim = 300
fasttext_dim = 300
sentence_embedding_dim = 300
sinbert_dim = 768  # Larger than others

# Apply parsing with dimension corrections
df['word2vec_vector'] = df['word2vec_vector'].map(lambda x: parse_vector(x, word2vec_dim))
df['fasttext_vector'] = df['fasttext_vector'].map(lambda x: parse_vector(x, fasttext_dim))
df['sentence_embedding'] = df['sentence_embedding'].map(lambda x: parse_vector(x, sentence_embedding_dim))
df['sinbert_vector'] = df['sinbert_vector'].map(lambda x: parse_vector(x, sinbert_dim))  # Keep 768D

# Ensure all feature vectors have the same final dimension
final_dim = 768  # Choose larger or normalize to 300
for feature in ['word2vec_vector', 'fasttext_vector', 'sentence_embedding']:
    df[feature] = df[feature].map(lambda x: np.pad(x, (0, final_dim - x.shape[0])) if x.shape[0] < final_dim else x[:final_dim])

# Stack feature vectors correctly
X = np.stack(df[['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']].apply(lambda row: np.column_stack(row.values), axis=1).values)

# Flatten X for MLP: (samples, final_dim * 4)
X_flattened = X.reshape(X.shape[0], -1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
joblib.dump(label_encoder, os.path.join(save_path, 'label_encoder.pkl'))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flattened, X_test_flattened = train_test_split(X_flattened, test_size=0.2, random_state=42)

# Function to implement early stopping and class weights
def build_mlp(input_shape, dropout_rate=0.2, dense_units=128):
    model = Sequential([
        Input(shape=input_shape),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(dense_units // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape, dropout_rate=0.2, lstm_units=64):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_shape, dropout_rate=0.2, filters=64):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters, kernel_size=3, activation='relu'),
        Dropout(dropout_rate),
        GlobalMaxPooling1D(),
        Dense(filters // 2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Apply class weights (use if there is class imbalance)
class_weights = {0: 1, 1: 1.5}  # Increase weight for the minority class

# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix.tolist()
    }

# Train and evaluate models with hyperparameter tuning and regularization
results = {}

# Model hyperparameters for tuning
params = {
    'MLP': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'dense_units': [64, 128]
    },
    'LSTM': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'lstm_units': [64, 128],
    },
    'CNN': {
        'epochs': [10, 20],
        'batch_size': [16, 32],
        'dropout': [0.2, 0.3],
        'filters': [64, 128],
    }
}

# Train and evaluate each model
for name in ['MLP', 'LSTM', 'CNN']:
    print(f"Training {name}...")

    # Hyperparameter grid search (manually tuned here)
    for epochs in params[name]['epochs']:
        for batch_size in params[name]['batch_size']:
            if name == 'MLP':
                for dropout in params[name]['dropout']:
                    for dense_units in params[name]['dense_units']:
                        model_instance = build_mlp((X_train_flattened.shape[1],), dropout, dense_units)
                        model_instance.fit(X_train_flattened, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test_flattened, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test_flattened) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'LSTM':
                for dropout in params[name]['dropout']:
                    for lstm_units in params[name]['lstm_units']:
                        model_instance = build_lstm((X_train.shape[1], X_train.shape[2]), dropout, lstm_units)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)
            elif name == 'CNN':
                for dropout in params[name]['dropout']:
                    for filters in params[name]['filters']:
                        model_instance = build_cnn((X_train.shape[1], X_train.shape[2]), dropout, filters)
                        model_instance.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                           validation_data=(X_test, y_test), class_weight=class_weights,
                                           callbacks=[early_stopping], verbose=1)
                        y_pred = (model_instance.predict(X_test) > 0.5).astype(int)
                        results[name] = evaluate_model(y_test, y_pred)

# Save results to JSON
results_path = os.path.join(save_path, "model_results.json")
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)

print("Experiment completed. Results saved.")